In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display
import csv
import json
import us

In [10]:
data_dir = Path.cwd().parent / 'DATA'

In [11]:
YEAR = '2020'
year_data = data_dir / YEAR

In [12]:
sources = {
    f_obj.name: f_obj 
    for f_obj in year_data.glob('*') 
    if f_obj.is_dir() and f_obj.name[0] != '.'
}
print(f"data sources: {list(sources.keys())}")

data sources: ['AHRQ', 'CDC_PLACES', 'CDC_SVI', 'EPA_AQI']


## Reference: County FIPS to County Name
Interactive page, plus multiple download formats  
<https://coastwatch.pfeg.noaa.gov/erddap/convert/fipscounty.html>

In [34]:
# load crosswalk
fips_xref = 'FipsCountyCodes'
fips_xref_js = data_dir / f"{fips_xref}.json"
with open(fips_xref_js, 'r') as js_in:
    fips_raw = json.load(js_in)['table']
print(fips_raw.keys())
fips_codes = fips_raw['rows']
print(f"code entries loaded: {len(fips_codes)}")

dict_keys(['columnNames', 'columnTypes', 'rows'])
code entries loaded: 3360


In [35]:
st_abbr_names = us.states.mapping('abbr', 'name')

In [36]:
states_terr = {}
counties = {}
n_skip, n_county, n_state = 0, 0, 0
for code, label in fips_codes:
    if ',' not in label:
        st_name = st_abbr_names.get(label, None)
        if st_name is None:
            print(f"skipping abbr: {label}, fips: {code}")
            n_skip += 1
        else:
            states_terr[label] = {
                'fips': code[0:2],
                'name': st_name,
            }
            n_state += 1
    else:
        if label in counties:
            print(f"found: {label}")
        counties[label] = code
        n_county += 1
print(f"loaded states/territories: {len(states_terr)}")
print(f"loaded counties: {len(counties)}")
print(f"counts: {n_skip=}, {n_county=}, {n_state=}")
print(f"check: {sum([len(states_terr), len(counties)]) == sum([n_county, n_state])}")

skipping abbr: FM, fips: 64000
skipping abbr: MH, fips: 68000
skipping abbr: PW, fips: 70000
skipping abbr: UM, fips: 74000
loaded states/territories: 55
loaded counties: 3301
counts: n_skip=4, n_county=3301, n_state=55
check: True


## NOTE: looking to merge data along US counties

In [16]:
source_files = {}
for src, src_dir in sources.items():
    print(src)
    for f_obj in src_dir.glob('*'):
        if 'COUNTY' in f_obj.name.upper() and 'GIS' not in f_obj.name.upper():
            if f_obj.suffix.upper() in ['.XLSX', '.CSV']:
                print(' ', f_obj.name)
                source_files[src] = {
                    'filename': f_obj.name,
                }
source_files

AHRQ
  SDOH_2020_COUNTY_1_0.XLSX
CDC_PLACES
  PLACES-Local-Data-for-Better-Health-County-Data.csv
  PLACES__Local_Data_for_Better_Health__County_Data_2020_release.csv
CDC_SVI
  SVI2020_US_COUNTY.csv
EPA_AQI
  daily_aqi_by_county_2020.csv


{'AHRQ': {'filename': 'SDOH_2020_COUNTY_1_0.XLSX'},
 'CDC_PLACES': {'filename': 'PLACES__Local_Data_for_Better_Health__County_Data_2020_release.csv'},
 'CDC_SVI': {'filename': 'SVI2020_US_COUNTY.csv'},
 'EPA_AQI': {'filename': 'daily_aqi_by_county_2020.csv'}}

In [17]:
KEYCOLS = ['COUNTY', 'FIPS', 'LOCATION', 'STATE', 'ST_']

In [18]:
for src, src_info in source_files.items():
    print(f"{src} loading...")
    keys = []
    src_file = year_data / src / src_info['filename']
    if src_file.suffix.upper() == '.XLSX':
        xl_file = pd.ExcelFile(src_file)
        src_data = pd.read_excel(xl_file, sheet_name='Data')
    elif src_file.suffix.upper() == '.CSV':
        src_data = pd.read_csv(src_file)
    n_rows, n_cols = src_data.shape
    print(f"data: {n_rows} rows, {n_cols} cols")
    src_info['data'] = src_data.copy()
    for col in src_data.columns:
        for keycol in KEYCOLS:
            if keycol in col.upper():
                keys.append(col)
                break
    if src == 'AHRQ':
        keys = [k for k in keys if '_' not in k]
    source_files[src]['key_cols'] = keys
    display(src_data[keys].head())
    print()

AHRQ loading...
data: 3229 rows, 682 cols


,COUNTYFIPS,STATEFIPS,STATE,COUNTY
0,1001,1,Alabama,Autauga County
1,1003,1,Alabama,Baldwin County
2,1005,1,Alabama,Barbour County
3,1007,1,Alabama,Bibb County
4,1009,1,Alabama,Blount County



CDC_PLACES loading...
data: 176008 rows, 21 cols


,StateAbbr,StateDesc,LocationName
0,US,United States,NaN
1,US,United States,NaN
2,US,United States,NaN
3,US,United States,NaN
4,US,United States,NaN



CDC_SVI loading...
data: 3143 rows, 158 cols


,STATE,ST_ABBR,COUNTY,FIPS,LOCATION
0,Alabama,AL,Autauga,1001,"Autauga County, Alabama"
1,Alabama,AL,Baldwin,1003,"Baldwin County, Alabama"
2,Alabama,AL,Barbour,1005,"Barbour County, Alabama"
3,Alabama,AL,Bibb,1007,"Bibb County, Alabama"
4,Alabama,AL,Blount,1009,"Blount County, Alabama"



EPA_AQI loading...
data: 324338 rows, 10 cols


,State Name,county Name,State Code,County Code
0,Alabama,Baldwin,1,3
1,Alabama,Baldwin,1,3
2,Alabama,Baldwin,1,3
3,Alabama,Baldwin,1,3
4,Alabama,Baldwin,1,3


## Normalize data on 5-digit State + County FIPS code (w/ leading 0)

In [74]:
indeces = {}

### CDC PLACES

In [44]:
# investigate CDC PLACES loactions
cdc_key_cols = source_files['CDC_PLACES']['key_cols']
cdc_keys = source_files['CDC_PLACES']['data'][cdc_key_cols].copy()

In [45]:
cdc_states = cdc_keys['StateAbbr'].unique()
print(f"unique states: {len(cdc_states)}", sorted(cdc_states))
print("-- 50 states plus 'US' & 'DC'")

unique states: 52 ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'US', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
-- 50 states plus 'US' & 'DC'


In [46]:
stnames_fips = {stnm:data['fips'] for stnm,data in states_terr.items()}

In [47]:
# normalize location names
cdc_keys['ST_FIPS'] = cdc_keys['StateAbbr'].map(stnames_fips)
cdc_keys['STATE_COUNTY'] = cdc_keys['StateAbbr'] + ', ' + cdc_keys['LocationName']
cdc_keys['STATE_COUNTY'] = cdc_keys['STATE_COUNTY'].replace('NM, DoÃ±a Ana', 'NM, Dona Ana')

In [48]:
# add county codes
cdc_keys['COUNTY_FIPS'] = cdc_keys['STATE_COUNTY'].map(counties)
cdc_keys['COUNTY_FIPS'] = cdc_keys['COUNTY_FIPS'].fillna('00000')

In [52]:
# check for missing codes
missing_codes = cdc_keys[cdc_keys['COUNTY_FIPS'].isnull()].copy()
print(f"missing county FIPS codes: {missing_codes.shape[0]}")

missing county FIPS codes: 0


In [50]:
missing_codes['STATE_COUNTY'].value_counts(dropna=False)

Series([], Name: STATE_COUNTY, dtype: int64)

In [63]:
cdc_keys[cdc_keys['StateAbbr'] != 'US'].head()

,StateAbbr,StateDesc,LocationName,ST_FIPS,STATE_COUNTY,COUNTY_FIPS
49,AL,Alabama,Autauga,01,"AL, Autauga",01001
50,AL,Alabama,Autauga,01,"AL, Autauga",01001
51,AL,Alabama,Barbour,01,"AL, Barbour",01005
52,KS,Kansas,Marion,20,"KS, Marion",20115
53,KS,Kansas,Osborne,20,"KS, Osborne",20141


In [75]:
indeces['CDC_PLACES'] = cdc_keys

### AHRQ

In [54]:
ahrq_key_cols = source_files['AHRQ']['key_cols']
ahrq_keys = source_files['AHRQ']['data'][ahrq_key_cols].copy()

In [58]:
ahrq_keys['COUNTY_FIPS'] = ahrq_keys['COUNTYFIPS'].astype('str').str.zfill(5)

In [65]:
# check for missing codes
missing_codes = ahrq_keys[ahrq_keys['COUNTY_FIPS'].isnull()].copy()
print(f"missing county FIPS codes: {missing_codes.shape[0]}")

missing county FIPS codes: 0


In [59]:
ahrq_keys.head()

,COUNTYFIPS,STATEFIPS,STATE,COUNTY,COUNTY_FIPS
0,1001,1,Alabama,Autauga County,01001
1,1003,1,Alabama,Baldwin County,01003
2,1005,1,Alabama,Barbour County,01005
3,1007,1,Alabama,Bibb County,01007
4,1009,1,Alabama,Blount County,01009


In [76]:
indeces['AHRQ'] = ahrq_keys

### CDC SVI

In [60]:
svi_key_cols = source_files['CDC_SVI']['key_cols']
svi_keys = source_files['CDC_SVI']['data'][svi_key_cols].copy()

In [61]:
svi_keys['COUNTY_FIPS'] = svi_keys['FIPS'].astype('str').str.zfill(5)

In [66]:
# check for missing codes
missing_codes = svi_keys[svi_keys['COUNTY_FIPS'].isnull()].copy()
print(f"missing county FIPS codes: {missing_codes.shape[0]}")

missing county FIPS codes: 0


In [62]:
svi_keys.head()

,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,COUNTY_FIPS
0,Alabama,AL,Autauga,1001,"Autauga County, Alabama",01001
1,Alabama,AL,Baldwin,1003,"Baldwin County, Alabama",01003
2,Alabama,AL,Barbour,1005,"Barbour County, Alabama",01005
3,Alabama,AL,Bibb,1007,"Bibb County, Alabama",01007
4,Alabama,AL,Blount,1009,"Blount County, Alabama",01009


In [82]:
indeces['CDC_SVI'] = svi_keys

### EPA AQI

In [64]:
epa_key_cols = source_files['EPA_AQI']['key_cols']
epa_keys = source_files['EPA_AQI']['data'][epa_key_cols].copy()

In [70]:
epa_keys['STATE_FIPS'] = epa_keys['State Code'].astype('str').str.zfill(2)
epa_keys['COUNTY_SHORT'] = epa_keys['County Code'].astype('str').str.zfill(3)
epa_keys['COUNTY_FIPS'] = epa_keys['STATE_FIPS'] + epa_keys['COUNTY_SHORT']

In [71]:
# check for missing codes
missing_codes = epa_keys[epa_keys['COUNTY_FIPS'].isnull()].copy()
print(f"missing county FIPS codes: {missing_codes.shape[0]}")

missing county FIPS codes: 0


In [72]:
epa_keys.head()

,State Name,county Name,State Code,County Code,STATE_FIPS,COUNTY_SHORT,COUNTY_FIPS
0,Alabama,Baldwin,1,3,01,003,01003
1,Alabama,Baldwin,1,3,01,003,01003
2,Alabama,Baldwin,1,3,01,003,01003
3,Alabama,Baldwin,1,3,01,003,01003
4,Alabama,Baldwin,1,3,01,003,01003


In [78]:
indeces['EPA_AQI'] = epa_keys

## Add County FIPS to Dataset and Output

In [95]:
MEGA = 1024*1024

In [94]:
for src, src_info in source_files.items():
    print(f"{src} data:")
    orig_df = src_info['data']
    print(f"> orig. cols: {orig_df.shape[1]}")
    cnty_fips = indeces[src]['COUNTY_FIPS']
    aug_df = orig_df.copy()
    aug_df['COUNTY_FIPS'] = cnty_fips
    print(f"> aug. cols: {aug_df.shape[1]}")
    src_datafile = year_data / src / f"{src}_county_raw.csv"
    aug_df.to_csv(src_datafile, index=False)
    print(f"records: {aug_df.shape[0]}")
    print(f"file size: {round(src_datafile.stat().st_size / MEGA, 2)} mb\n")

AHRQ data:
> orig. cols: 682
> aug. cols: 683
records: 3229
file size: 10.54 mb

CDC_PLACES data:
> orig. cols: 21
> aug. cols: 22
records: 176008
file size: 42.16 mb

CDC_SVI data:
> orig. cols: 158
> aug. cols: 159
records: 3143
file size: 2.2 mb

EPA_AQI data:
> orig. cols: 10
> aug. cols: 11
records: 324338
file size: 21.76 mb

